# Assignment 5: A Naïve Bayes Classifier
## CSCI2244-Randomness and Computation
## Due , March 26, 11:59PM

__Your Name:__

This assignment has you implement a  Naïve Bayes Classifier, a machine learning tool based on Bayes's Theorem. The theory behind this assignment is explained in the Examples section, especially in 1.2.  The specific task for you to carry out is described in the subsequent sections.
  

## 1. Some examples to warm up

### 1.1 Coins


> Imagine that you have a job as a coin-tester.  You know that the coins you are given to test are either unfair coins with heads probability ${1\over 4},$ unfair coins with heads probability ${3\over 4},$ and fair coins.  You are given a coin to test.  At the outset, you have no reason to believe that the coin is more likely to be of one type than of either of the other types.   You then flip the coin seven times and get the outcome HHTHTTH.  As a result of this (and Bayes's Theorem), you re-calibrate your beliefs and assign new probabilities to each of the three alternatives.  What are they? 

Here is a solution.  Let $A,B,C$ denote, respectively the event that the coin we are handed has heads probability $1\over 4,$ $3\over 4,$ or $1\over 2.$ Let $HHTHTTH$ denote the event that this particular sequence of 7 tosses is made.  We're asked to determine the conditional probabilities

$$P(A|HHTHTTH),P(B|HHTHTTH),P(C|HHTHTTH).$$

By Bayes's Theorem,

$$P(A|HHTHTTH)={{P(HHTHTTH|A)P(A)}\over {P(HHTHTTH)}}.$$

$P(A)={1\over 3},$ since we think the three kinds of coins are equally likely. By our usual methods,

$$P(HHTHTTH|A)= \biggl ({1\over 4}\biggr)^4\biggl({3\over 4}\biggr)^3 ={{27}\over {4^7}}.$$

So 

$$P(A|HHTHTTH)={{{27}\over {3\cdot 4^7}} \over {P(HHTHTTH)}}.$$

Likewise,

$$P(B|HHTHTTH)={{{81}\over {3\cdot 4^7}} \over {P(HHTHTTH)}},$$

and

$$P(C|HHTHTTH)={{{1}\over {3\cdot 2^7}} \over {P(HHTHTTH)}}= {{{128 }\over {3\cdot 4^7}} \over {P(HHTHTTH)}}.$$

Since the denominators are all the same, we don't have to evaluate them; we know that the three newly assigned probabilities are in the proportions 27,81,128, with $C$ most likely.

### 1.2 M&M's

You receive samples of M&M's from two different factories, and find that the distribution of 
colors is as follows:

 | | Factory 1 | |  Factory 2  | | 
 |---|---|---|---|---| 
 |__Color__|__Absolute__|__Relative__|__Absolute__|__Relative__|
 |__Red__|84|0.456|60|0.255| 
 |__Brown__|56 |0.304 |40 |0.170|  
 |__Green__|24 |0.130| 40| 0.170  |
 |__Yellow__| 0| 0 |10| 0.043 | 
 |__Blue__| 20| 0.109| 85| 0.362| 
 |__Total__|184|1.00|235|1.00| 
  

This is the _training data_.  You then receive a package of M & M's of unknown origin and find that the colors are distributed as follows:

 |Color| Quantity|
 |---|---|
 |__Red__| 18 | 
 |__Brown__| 11|   
 |__Green__| 10 |   
 |__Yellow__| 4|   
 |__Blue__| 1 |  
 |__Orange__| 2|  
 |__Total__| 46 |


The classifier is supposed to use this information and the training data to determine whether the new package is from Factory I or Factory II.  

We  think of the first table as giving conditional probabilities. For example
$$p_{RI}=P(R|I)=0.456$$
is the probability that an M&M from Factory I is red. We will also write $P(RBRG|I)$ and the like for the probability of selecting four M & M's from Factory I and getting a red, blue, red and green M&M in sequence.  We then have
$$P(RBRG|I)=p_{RI}^2p_{GI}p_{BI}.$$
The new package of M&M's is viewed as the outcome $E$ of an experiment in which we select a sequence of 46 candies from one of the two factories and find 18 Reds, 11 Browns, 10 Greens, 4 Yellows, 1 Blue, and 1 Orange.  We want to determine $P(I|E)$ and $P(II|E).$ The larger of these values will be our determination of whether the package came from Factory I or Factory II.  Let's suppose that we also have some prior knowledge:  that Facory II has three times the output of Factory I, so that $P(II)={3\over 4},$ $P(I)={1\over 4}.$

So we apply Bayes's Theorem and find

$$P(I|E)={{P(E|I)P(I)}\over {P(E)}}={{0.75\cdot p_{RI}^{18}p_{BrI}^{11}p_{GI}^{10}p_{YI}^4p_{BlI}p_{OI}^2}\over{P(E)}},$$
and similarly for $P(II|E).$ The denominators are the same, so all we have to do is multiply everything in the two numerators and compare the results, but...

#### 1.2.1 Add-1 smoothing

...there's a problem. There are no yellow M&M's in the training data from Factory I, so we will automatically assign a probability of 0 to $P(I|E)$ if the package we're inspecting contains a blue M&M.  But it may just happen that yellow M&M's are rare, although not impossible, outputs of Factory I, and that evidence from the distribution of the other colors should tip the scales to Factory I. And there are no orange M&M's in the training data from either factory, so in fact in this example, both $P(I|E)$ and $P(II|E)$ are assigned 0.

We handle this problem as follows:  We simply skip the orange M&M's when we  compute the probabilities.  To make sure that we get a positive, although small, probability, for values that occur in only one of the two samples, we tweak the table by adding one to _all_ the counts, and get a revised table, which leads to revised estimates of the probabilities $p_{RI},$ _etc._ .

 | | Factory 1 | |  Factory 2  | | 
 |---|---|---|---|---| 
 |__Color__|__Absolute__|__Relative__|__Absolute__|__Relative__|
 |__Red__|85|0.450|61|0.254| 
 |__Brown__|57 |0.302 |42 |0.175|  
 |__Green__|25 |0.132| 42| 0.175  |
 |__Yellow__|1| 0.0053 |11| 0.046 | 
 |__Blue__| 21| 0.111| 87| 0.3625| 
 |__Total__|189|1.00|240|1.00| 
 
 Now we can carry out our computation of the numerators in $P(I|E)$ and $P(II|E).$. For the first we get
 
 $${0.75\cdot p_{RI}^{18}p_{BrI}^{11}p_{GI}^{10}p_{YI}^4p_{BlI}p_{OI}^2}=0.75\cdot 0.45^{18}\cdot 0.302^{11}\cdot 0.132^{10}\cdot 0.0053^4\cdot 0.111,$$
 and for the second
 
 $$0.25\cdot 0.254^{18}\cdot 0.175^{11}\cdot 0.175^{10}\cdot 0.046^4\cdot 0.3625.$$
 
 #### 1.2.2 Avoiding underflow
 
 These are _very_ small numbers, about $10^{-31}$ and $10^{-33}$ respectively.  Since the first is quite a bit larger, we conclude that our packet likely came from Factory I.  But in other applications of the same technique, the numbers could be even smaller, and lead to _floating point underflow_: values that are too small to be represented as floating point numbers.  So it is a common practice to take the _logarithms_ of these products, and compare them instead.  So for the above example we would have
 
 $$\ln 0.75 +18\cdot\ln 0.45+11\cdot\ln 0.302 +10\cdot\ln 0.132+4\cdot\ln 0.046+\ln 0.3625,$$
 
 about -71.3, and similarly for the second, which gives -75.98. Since $-71.3>-75.98,$ we again conclude that the new pack probably came from Factory I. In the present example, we did not need to carry out this last step, because we did not get underflow, but in examples where there is more data to process, it is essential. 
 

 
 
 
## 2. Your mission

In this problem, you will implement a Naïve Bayes Classifier to distinguish SMS spam messages.

The dataset consists of 5574 SMS messages, of which 747 have been labeled (by humans!) as spam  and the other 4827 as non-spam, for which the term of art, it seems, is 'ham'.  (I obtained the file from the website Kaggle, but it originates from researchers in Brazi whose website appears to be no longer active. You can find a more detailed description of the dataset here.)

The original data, in `spam2.csv`is posted on Canvas site, so you can take a look at it, if you care to. (Warning:  Not surprisingly, with a large collection of text messages, many of them spam, there is a fair amount of creepy or offensive content.)  You do not need to look at his, however; I have done most of the pre-processing dirty, and provided you with four text files:

```
spam_sms_training
ham_sms_training
spam_sms_test
ham_sms_test
```

Each of these files contains one message per line, purged of punctuation and 'stop words' like 'the', 'and', etc, and converted into lower case.  The first contains half of the messages from the original dataset that were labeled as spam, the second half of the messages that were labeled as ham.  The other halves  are in the files with `test` in the name.  These are the four files that you will work with. 

The goal is to use the training files to build models giving the likely distributions of words in spam and ham messages, then to use these models to classify the messages in the test files, using the Naïve Bayes method.  You will submit the code you wrote, and report on how well the classifier worked.

### 2.1 The theory



If you choose a word at random from a spam message, what is the probability that this word is, say, 'prize'? We write this as
$$P(\text{'prize'}|{\tt spam}).$$
In doing so, we conceive of spam as a giant 'bag of words', in which each word appears with a particular probability, much like the urns of M & M's described in the lecture notes.  And we think of ham as another such bag. 

A message $D$ is a sequence of words
$$D=(w_1,\ldots,w_r).$$
We think of $D$ as being generated by reaching into one of the bags $r$ times, and pulling out the word $w_i$ on the $i^{th}$ draw.  Thus
$$P(D|{\tt spam})=\prod_{i=1}^rP(w_i|{\tt spam}).$$
This is the naïve part of Naïve Bayes---in effect we're treating 'you have won a prize' as just as likely to be generated as a spam message as prize a won have you', and ignoring anything related to the order of words or occurrence of certain multi-word phrases.  While this assumption is obviously false, in practice it leads to effective results.

The task of classification is to take a message $D$ and decide whether it is spam or not; that is, we want to decide which of the two values
$$P(\text{spam}|D),\hspace{0.1in} P(\text{ham}|D)$$
is larger.  By Bayes's Theorem,
$$P(\text{spam}|D)={{P(D|\text{spam})}\over {P(D)}}\cdot P(\text{spam}),\hspace{0.1in} P(\text{ham}|D)={{P(D|\text{ham})}\over {P(D)}}\cdot P(\text{ham}).$$
We can ignore the common denominator $P(D)$ on the right-hand sides, and apply the naïve assumption above, so that the problem now is to determine which of the following is larger:
$$P(\text{apam})\cdot \prod_{i=1}^rP(w_i|\text{spam}),\hspace{0.1in} P(\text{ham})\cdot \prod_{i=1}^rP(w_i|\text{ham}).$$
We call these two numbers the 'spam score' and the 'ham score' of the document $D.$

### 2.2 Estimating probabilities

To compute the scores, we need to have estimates for $P(w|{\tt spam})$ and $P(w|{\tt ham})$ for each word $w,$  as well as estimates of $P({\tt spam})$ and $P({\tt ham}).$ The simplest idea is to gather up all the spam messages in our training data, count the total number $N_{\tt spam}$ of words, and the total number $N_{w,{\tt spam}}$ of occurrences of $w$ among these words, and estimate
$$P(w|\text{spam})\approx {{N_{w,{\text{spam}}}\over {N_{\tt spam}}}}.$$
For example, if there are a total of 6000 words  in the spam messages in the training data, and 150 of them are the word 'call', then we would approximate
$$P(\text{`call'}|\text{spam})\approx 150/6000=0.025.$$

Of course, we do the same thing to estimate $P(w|\text{ham}).$

We also need to have estimates of $P(\text{spam})$ and $P(\text{ham}).$  These are our 'priors'.  Here we require some kind of global estimate of the fraction of messages that are spam. For purposes of this problem, you can use the proportion of spam messages in the data set, which is about 13.4%.  So we can estimate $P(\text{spam})\approx 0.134,$ $P(\text{ham})\approx 0.866.$

### 2.3 Add-1 smoothing

What if a word, say, 'excite', appears once or twice in the ham training set, but never in the spam training set? $P(\text{`{\tt excite}'}|{\tt spam})$ will be then be estimated as 0, while  $P(\text{`{\tt excite}'}|{\tt ham})$ will be positive.  If we then try to evaluate a document that happens to be spam and contains this word, then the spam score will be 0, but the ham score might be non-zero, so the document will be classified incorrectly as ham. To get around this, we artificially add 1 to every count, pretending that every word encountered in the training sets appears in both ham and spam documents:  Let $V$ be the set of distinct words appearing in the two training sets, either in ham or in spam, and set for each $w\in V,$

$$P(w|\text{spam})\approx {{{N_{w,\text{spam}}}+1}\over {N_{\text{spam}+|V|}}},$$
and likewise for ham.  If the document $D$ contains words that appear in neither training set, then we just skip these words in computing the score.  In this manner, every document will get a positive spam score and a positive ham score.

### 2.4 Taking logs to avoid underflow
 
 Typically, for any given word $w,$ $P(w|{\tt spam})$ and $P(w|{\tt ham})$ will be quite small, in the neighborhood of $10^{-4}$ or less. If a document contains $r>100$ words, we will wind up risking floating-point underflow when we try to evaluate the product
 $$P({\tt spam})\cdot \prod_{i=1}^rP(w_i|{\tt spam}).$$
 To avoid this, we take the logarithm of the product, which is the sum of the respective logarithms, and replace the score by
 $$\ln(P({\tt spam}))+\sum_{i=1}^r\ln(P(w_i|{\tt spam})).$$
 Note that this will be a negative number, since we are taking logs of numbers less than 1.  If you want, you can negate it and make the score positive, but just keep in mind that if you do this,  you will want to find which of spam or ham gives a _lower_ score.



## 3. What you will write

Write a function `build_models` that returns a pair of dictionaries, one giving the smoothed probabilities from the spam messages used for training, and the ham messages used for training. Each item in each dictionary is a key-value pair in which the word is the key and its probability is the value.) Remember that after you compute the initial counts, you need to tweak them so that both dictionaries contain the same keys, and then change absolute counts to relative counts.

Note that since we have removed punctuation, it is easy to extract the list of words from a line `s` of text: just write `s.split()`.


In [75]:
def build_models():
    # create list of probabilities for spam
    spam_probabilities = {} 
    
    # count for total words in spam
    spam_N = 0
    
    # open spam file
    f = open("spam_sms_training.txt",'r') 
    
    # populate dictionary with words and their frequency, increment count 
    for line in f: 
        for word in line.split(): 
            spam_N += 1
            if word in spam_probabilities: 
                spam_probabilities[word] += 1 
            else: 
                spam_probabilities[word] = 1 
                
    # create list of probabilities for ham 
    ham_probabilities = {}
    
    # count for total words in ham 
    ham_N = 0
    
    # open ham file 
    f2 = open("ham_sms_training.txt", 'r') 
    
    # populate dictionary with words their frequency, and increment total count 
    for line in f2: 
        for word in line.split(): 
            ham_N += 1
            if word in ham_probabilities: 
                ham_probabilities[word] += 1 
            else: 
                ham_probabilities[word] = 1 
    
    # make keys the same 
    for key in spam_probabilities:
        if key not in ham_probabilities:
            ham_probabilities[key] = 0
            

    for key in ham_probabilities: 
        if key not in spam_probabilities: 
            spam_probabilities[key] = 0
    
    # get number of distinct keys  
    v = len(spam_probabilities)
    
            
    # calculate probability by dividing total count + count of words that are in one but not other 
    # add one for smoothing 
    for key in spam_probabilities: 
        spam_probabilities[key] = (spam_probabilities[key] + 1)/(spam_N + v)
    
    for key in ham_probabilities: 
        ham_probabilities[key] = (ham_probabilities[key] + 1 )/(ham_N + v)        
    
    return spam_probabilities, ham_probabilities



 
 Then write a function `score(message,model,prior)` that takes a message and computes its score relative to the model, which will be one of the dictionaries returned by `build_models`. The scoring should include a prior probability. (See above for how to estimate this).

In [76]:
import math 

def score(message, model, prior):
    
    probability_total = 0
    
    # iterate through message find values in the model 
    for x in message.split(): 
        
        if x in model: 
            # if in the model then sum probability and ln 
            probability_total = probability_total + math.log(model[x]) 
            
    return (math.log(prior) + probability_total) 

 
 Finally, write a function `evaluate()` that evaluates the test data.  One suggestion for how to do this--load all the ham test messages and all the spam test messages into a single list.  Then repeatedly, say one thousand times, pick a message at random and use your classifier to classify it as spam or ham.  `evaluate` should return a tuple `(tp,tn,fp,fn,accuracy,precision,recall)` as described below.
 

 


In [77]:
import random 
def evaluate():
    
    # create variables 
    tp = 0 
    tn = 0 
    fp = 0 
    fn = 0 
    messages = [] 
    
    # open spam test messages
    f = open("spam_sms_test.txt",'r') 
    
    # append them to list and get last index of spam message 
    for line in f:
        line = line.strip()
        messages.append(line) 
    spam_range = len(messages) 

    # opem ham test messages 
    f2 = open("ham_sms_test.txt", 'r') 
    
    # append ham messages to list 
    for line in f2: 
        line = line.strip()
        messages.append(line)
        
    spam, ham = build_models()
        
    # for 1000 times get the index which identifies if spam or ham, calculate score, compare 
    for x in range(1000): 
        rand_index = random.randint(0,len(messages)-1)
   
        spam_score = score(messages[rand_index], spam, .134) 
    
        ham_score = score(messages[rand_index], ham, .866) 
    
        if spam_score > ham_score: 
            if rand_index in range(spam_range): 
                tp += 1
            else: 
                fp += 1 
        else: 
            if rand_index in range(spam_range): 
                fn += 1
            else: 
                tn += 1 
                
    accuracy = (tp + tn) / (tp + fp + tn + fn) 
    precision = tp / (tp + fp)
    recall = tp / (tp + fn) 
    
    return tp, fp, fn, tn, accuracy, precision, recall; 

 How should you do this evaluation?  Treat the smaller of the two classes (in thi case, spam) as the 'rare trait' we are testing for.  Thus we treat a spam classification as 'positive' and a ham classification as negative.  As we run the classifier on each item in the test set, we compare the classifier result to the correct answer, and keep four counts:
 
 * $TP$--the number of true positives.  This will be the number of spam messages that the classifier says are spam.
 
 * $TN$--the number of true negatives.  This will be the number of ham messages classified as such.
 
 * $FP$--the number of false positives:  ham  classified as spam.
 
* $FN$--the number of false negatives: spam classified as ham.
 
 
 It is a common practice in machine learning to employ a number of different statistics to evaluate the performance of this kind of binary classifier.  An obvious one is the _accuracy,_ the proportion of answers that are correct:
 
 $$\text{accuracy}={{TP+TN}\over{TP+FP+TN+FN}}.$$
 As our drug testing example shows, accuracy is not the whole story. For instance, a diagnostic test for a very rare disease that consists in saying 'You don't have it!' to every subject, could be 99.9% accurate, but it would fail to diagnose any actual instance of the disease that it encountered.  For this reason, other measures are also computed.  The _precision_ is the proportion of positive test results that are correct.
 
  $$\text{precision}={{TP}\over{TP+FP}}.$$
  This was the failing in our example of testing senior citizens for marijuana use:  The accuracy was high, but the precision was only around 50\%.  The _recall_ is the proportion of positive instances that are correctly diagnosed:
  
   $$\text{recall}={{TP}\over{TP+FN}}.$$
  The 'accurate' diagnostic test that always answers negative has zero recall!



In [78]:
#The main program is just a wrapper


def NaiveBayes():
     
    tp,fp,fn,tn,accuracy,precision,recall=evaluate()
    print("number of true positives: ",tp)
    print("number of true negatives: ",tn)
    print("number of false positives: ",fp)
    print("number of false negatives: ",fn)
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    

    

In [79]:
NaiveBayes()

number of true positives:  112
number of true negatives:  869
number of false positives:  4
number of false negatives:  15
accuracy:  0.981
precision:  0.9655172413793104
recall:  0.8818897637795275


How well did your classification perform?  Give a brief description of your results below:

suprisingly the classification performed very accurately with high precision and relatively high recall. 